In [1]:
#!pip install georasters
#!pip install geoindex
#!pip install spark
#!pip install pyspark

In [2]:
import pandas as pd
import georasters as gr
import matplotlib.pyplot as plt
from geoindex import GeoGridIndex, GeoPoint
import os
from tqdm import tqdm, tqdm_pandas

In [3]:
pd.set_option('display.max_columns', 100)

## Load Water Data

In [4]:
os.getcwd()

'/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access/Population'

In [5]:
water_data = pd.read_csv("/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access//data/Water_Point_Data_Exchange_Complete_Dataset.csv")

/Users/alexjamar/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,5,7,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
water_data.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,NaN,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,SNV,Arusha,Longido,wpdx-00000001,01/05/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.83257,36.3090,"(-2.83257, 36.309)",1,NaN,NaN,NaN
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,SNV,Arusha,Longido,wpdx-00000002,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84781,36.5284,"(-2.84781, 36.5284)",1,NaN,NaN,NaN
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000003,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84796,36.5288,"(-2.84796, 36.5288)",1,NaN,NaN,NaN
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000004,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.85318,36.5276,"(-2.85318, 36.5276)",1,NaN,NaN,NaN
4,5,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,El Porvenier,Matagalpa,Terrabona,wpdx-00000005,01/01/2011 12:00:00 AM +0000,NI,NaN,http://www.dropbox.com/s/0f70mln5roxhy29/ProjT...,https://www.waterpointdata.org/dataset/resourc...,NaN,"#data_lnk, #lon_deg, #lat_deg, #country_id",04/08/2016 08:35:49 AM +0000,04/08/2016 08:35:49 AM +0000,12.74070,-86.0121,"(12.7407, -86.0121)",1,NaN,NaN,NaN


In [7]:
uganda_water = water_data[water_data['#country_name'] == 'Uganda']

In [8]:
uganda_water.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
38495,58112,Uganda,NaN,Non-Mechanized Borehole,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058112,08/06/2012 12:00:00 AM +0000,UG,Kiranganzi,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,1.33007,31.3906,"(1.33007, 31.3906)",1,NaN,NaN,NaN
38496,58113,Uganda,NaN,Non-Mechanized Borehole,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058113,07/05/2012 12:00:00 AM +0000,UG,Njarwe (DWD 37910),http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,1.31209,31.3887,"(1.31209, 31.3887)",1,NaN,NaN,NaN
38497,58114,Uganda,NaN,Non-Mechanized Borehole,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058114,07/13/2012 12:00:00 AM +0000,UG,Ruhunga Primary School (DWD 37905),http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,1.23417,31.2732,"(1.23417, 31.2732)",1,NaN,NaN,NaN
38498,58115,Uganda,NaN,Non-Mechanized Borehole,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058115,07/24/2012 12:00:00 AM +0000,UG,Amwa Health center,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,2.31964,32.3859,"(2.31964, 32.3859)",1,NaN,NaN,NaN
38499,58116,Uganda,NaN,Non-Mechanized Borehole,yes,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058116,07/24/2012 12:00:00 AM +0000,UG,Bar,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,2.43701,32.4062,"(2.43701, 32.4062)",1,NaN,NaN,NaN


## Radius-based Search

In [9]:
uganda_pop = pd.read_csv("Uganda_pop_data.csv")

In [10]:
#Create Geo Index of Uganda Population Data
geo_index = GeoGridIndex()
for index, row in uganda_pop.iterrows():
    geo_index.add_point(GeoPoint(row['y'], row['x'], ref=row['value']))

In [11]:
#Create Geo Index of Uganda Water Point Data
geo_index_water = GeoGridIndex()
for index, row in uganda_water.iterrows():
    try:
        geo_index_water.add_point(GeoPoint(row['#lat_deg'], row['#lon_deg'], ref=row['#status_id']))
    except:
        print("Row ID:", row['Row ID'])
        print("Lat:", row['#lat_deg'])
        print("Long:", row['#lon_deg'])

Row ID: 393065
Lat: 330.248
Long: 30.9363
Row ID: 393703
Lat: 330.248
Long: 30.9363


In [12]:
#Calculate population within x distance of water point
def calculate_population_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_population = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_population += point.ref
    except:
        print("Invalid data - Record skipped")
    #print("Total population within", km, "kilometers:", int(total_population))
    return total_population

In [ ]:
#Calculate # of water points within x distance of water point
def calculate_water_points_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_water_points = 0
    total_functioning_water_points = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_water_points += 1
            if point.ref == 'yes':
                total_functioning_water_points += 1
    except:
        print("Invalid data - Record skipped")
        
    #Adjust for self
    total_water_points -= 1
    if row['#status_id'] == 'yes':
        total_functioning_water_points -= 1
    return (total_water_points, total_functioning_water_points)

In [ ]:
#Add 1 km radius population data to each well
tqdm.pandas()
uganda_water['1 km population'] = uganda_water.progress_apply(lambda row: calculate_population_within_x_km(row, geo_index, 1),
                                                  axis = 1)

 63%|██████▎   | 71512/113667 [76:33:40<50:50:45,  4.34s/it] 

In [ ]:
#Add 1 km radius water point data to each water point
tqdm.pandas()
uganda_water['1 km water points (total, functioning)'] = uganda_water.progress_apply(lambda row: calculate_water_points_within_x_km(row, geo_index_water, 1),
                                                  axis = 1)

In [ ]:
uganda_water.head(2)

In [ ]:
def clean_water_data(df):
    df['1 km population'] = df['1 km population'].apply(lambda x: int(round(x,0)))
    df['1 km total water points'] = df['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[0][1:])
    df['1 km functioning water points'] = df['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[1][:-1])
    df = df[['Row ID', '#country_name', '#status_id', '#lat_deg', '#lon_deg', '1 km population',
                        '1 km total water points', '1 km functioning water points']]
    
    return df

In [ ]:
uganda_abridged = clean_water_data(uganda_water)

In [ ]:
uganda_abridged.head()

In [19]:
afghanistan_abridged.to_csv("Abridged_Afghanistan_Water_and_Population_Data.csv", index = False)